In [ ]:
class Expert_Group():
    '''
    Experts are represented at the group level, questions of belief merging and aggregation 
    within the group are abstracted. individual experts are simulated for fun.

    Expert groups are static in this model. Their credences do not update

    Expert groups are defined relative to an ideal credence function (since the attribute "bias" is relative to ideal credence)
    '''
    def __init__(self, ideal_credence_obj:Ideal_Credences, name = None, simulation_sample_size:int = 1000, initial_competence:float = 0.2, initial_consensus:float = 0.2, initial_bias:float = 0.1) -> None:
        if name is not None:
            self.name = name
        else:
            self.name = name_generator()
        # simulation settings
        self.sample_size = simulation_sample_size
        # each expert group is defined by three attributes, competence (precision), bias (accuracy), and consensus (variance of expert judgements)
        self.competence = initial_competence
        self.bias = initial_bias
        self.consensus = initial_consensus
        self.ideal_credences = ideal_credence_obj.credence_function
        self.propositions = list(self.ideal_credences.keys())
        # each expert group has a set of aggregate judgements, which are functions that map credences to propositions
        self.aggregate_judgements = {} 
        self.get_aggregate_judgements()


    def get_aggregate_judgements(self):
        ''' take in a list of propositions'''
        keys = self.propositions
        values = np.array(list(self.ideal_credences.values()))
        
        # Apply bias and add randomness with specified standard deviation
        aggregate_judgement_values = np.random.normal(values + self.bias, self.competence, len(values))
        
        # Clip values to be within [0, 1]
        aggregate_judgement_values_clipped = np.clip(aggregate_judgement_values, 0, 1)
        
        self.aggregate_judgements = dict(zip(keys, aggregate_judgement_values_clipped))

    
    def simulate_expert_individual_judgements(self, plot = False):
        # get a dictionary of numpy arrays centered around the expert's aggregate judgement. mostly for fun and visualization
        # assumes a very naive view of belief aggregation.
        self.individual_judgements = {prop: np.clip(np.random.normal(loc=val, scale=self.consensus, size=self.sample_size), 0, 1)
                   for prop, val in self.aggregate_judgements.items()}
        if plot:
            self.plot_individual_judgements()

    
    def plot_individual_judgements(self):
        """
        Plots the samples for each proposition in self.aggregate_judgements.
        Each proposition's samples are plotted as a histogram.
        """
        n_props = len(self.propositions)
        label = f'{self.name} (Bias: {self.bias}, Consensus: {self.consensus}, Competence: {self.competence})'
        plt.figure(figsize=(12, 3 * n_props))

        for i, (prop, values) in enumerate(self.individual_judgements.items(), start=1):
            plt.subplot(n_props, 1, i)
            sns.histplot(values, kde=True, bins='auto', color='skyblue', label=label, stat="density")
            plt.axvline(x=self.aggregate_judgements[prop], color='r', linestyle='--', label=f'Aggregate judgement')
            plt.axvline(x=self.ideal_credences[prop], color='b', linestyle='--', label=f'Ideal credence')
            plt.legend()
            plt.title(f'Samples Distribution for {self.name} judgements on "{prop}"')
            plt.xlabel('Value')
            plt.xlim(0,1)
            plt.ylabel('Frequency')

        plt.tight_layout()
        plt.show()



#### Questions
        '''
        1. should expert group attributes be relativised to single beliefs? if so, 
        it makes it hard to capture how distrust spreads to other propositions.
        2. The truth + bias parameter should determine the aggregate judgement of experts on 
        every single proposition. This seems problematic. shouldn't it vary? But bias on 
        different propositions should have a strong correlation with each other too.
        '''